In [35]:
import numpy as np
import cv2

images = [cv2.imread('data/hyu_{}.jpeg'.format(i)) for i in range(3)]
grays = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in images]


sift = cv2.xfeatures2d.SIFT_create()
kps = []
descs = []
for gray in grays:
    kp, desc = sift.detectAndCompute(gray, None)
    kps.append(kp)
    descs.append(desc)

bf = cv2.BFMatcher()
matches1 = bf.match(descs[0], descs[1])     # between image 0 and image 1
matches2 = bf.match(descs[2], descs[1])     # between image 2 and image 1

In [ ]:

sorted_matches = sorted(matches1, key = lambda x : x.distance)
res = cv2.drawMatches(images[0], kps[0], images[1], kps[1], sorted_matches[:50], None, flags = 2)

from PIL import Image
Image.fromarray(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))

In [ ]:
sorted_matches = sorted(matches2, key = lambda x : x.distance)
res = cv2.drawMatches(images[2], kps[2], images[1], kps[1], sorted_matches[:50], None, flags = 2)

from PIL import Image
Image.fromarray(cv2.cvtColor(res, cv2.COLOR_BGR2RGB))

In [ ]:
height, width, _ = images[0].shape
result = np.zeros((height, width * 3, 3), dtype = np.uint8)

# Stitch image2 to image1
src = np.float32([kps[2][m.queryIdx].pt for m in matches2]).reshape((-1, 1, 2))
dst = np.float32([kps[1][m.trainIdx].pt for m in matches2]).reshape((-1, 1, 2))
H, status = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)

# Prepare pixel coordinates in image1
before = []
for x in range(images[1].shape[1]):
    for y in range(images[1].shape[0]):
        point = [x, y, 1]
        before.append(point)
before = np.array(before).transpose()

# [TODO] Calculate coordinates for each pixel in image1 to image 2
after = np.dot(H, before)
after = after / after[2, :]
after = after[:2, :]
after = np.round(after, 0).astype(int)


for pt1, pt2 in zip(before[:2, :].transpose(), after.transpose()):
    if pt2[1] >= height:
        continue

    if np.sum(pt2 < 0) >= 1:
        continue

    result[pt2[1], width+pt2[0]] = images[2][pt1[1], pt1[0]]


# Stitch image0 to image1
src = np.float32([kps[0][m.queryIdx].pt for m in matches1]).reshape((-1, 1, 2))
dst = np.float32([kps[1][m.trainIdx].pt for m in matches1]).reshape((-1, 1, 2))
H, status = cv2.findHomography(src, dst, cv2.RANSAC, 5.0)


# Prepare pixel coordinates in image1
before = []
for x in range(images[1].shape[1]):
    for y in range(images[1].shape[0]):
        point = [x, y, 1]
        before.append(point)
before = np.array(before).transpose()

# [TODO] Calculate coordinates for each pixel in image1 to image 0
after = np.dot(H, before)
after = after / after[2, :]
after = after[:2, :]
after = after.astype(int)
after[0] += width

for pt1, pt2 in zip(before[:2, :].transpose(), after.transpose()):
    if pt2[1] < 0 or pt2[1] >= height:
        continue

    if any(pt2 < 0):
        continue

    result[pt2[1], pt2[0]] = images[0][pt1[1], pt1[0]]
result[0:height, width:width*2] = images[1]

# Homography calculation
def calculate_homography(matches, kp_src, kp_dst):
    src_pts = np.float32([kp_src[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp_dst[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return H

H0 = calculate_homography(matches1, kps[0], kps[1])
H2 = calculate_homography(matches2, kps[2], kps[1])

# Determine output size
def get_output_size(images, H0, H2):
    h1, w1 = images[1].shape[:2]
    corners = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)

    # Warp corners for all images
    corners_0 = cv2.perspectiveTransform(corners, H0)
    corners_2 = cv2.perspectiveTransform(corners, H2)
    all_corners = np.concatenate((corners, corners_0, corners_2), axis=0)

    # Find bounding box
    x_min, y_min = np.int32(all_corners.min(axis=0).ravel())
    x_max, y_max = np.int32(all_corners.max(axis=0).ravel())

    return x_min, y_min, x_max, y_max

x_min, y_min, x_max, y_max = get_output_size(images, H0, H2)
output_size = (x_max - x_min, y_max - y_min)

# Warp images
offset = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])
H0_offset = offset @ H0
H2_offset = offset @ H2

result = np.zeros((output_size[1], output_size[0], 3), dtype=np.uint8)
cv2.warpPerspective(images[0], H0_offset, output_size, dst=result, borderMode=cv2.BORDER_TRANSPARENT)
cv2.warpPerspective(images[2], H2_offset, output_size, dst=result, borderMode=cv2.BORDER_TRANSPARENT)

# Insert the middle image (image 1)
x_offset = -x_min
y_offset = -y_min
result[y_offset:y_offset + images[1].shape[0], x_offset:x_offset + images[1].shape[1]] = images[1]

# Feather blending
def feather_blend(img1, img2):
    mask1 = (img1 > 0).astype(np.float32)
    mask2 = (img2 > 0).astype(np.float32)
    blended = (img1 * mask1 + img2 * mask2) / (mask1 + mask2 + 1e-5)
    return np.clip(blended, 0, 255).astype(np.uint8)

# Final blending
final_result = feather_blend(result, result)

# Show result
Image.fromarray(cv2.cvtColor(final_result, cv2.COLOR_BGR2RGB))



In [ ]:
from PIL import Image
Image.fromarray(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))